# Elliptic Curve Integrated Encryption Scheme

In [1]:
from typing import Tuple
from Crypto.Cipher import AES
import base64
import random

In [2]:
def is_on_curve(P: Tuple[int, int], p: int):
    x, y = P
    assert ( (a*x*x)+(y*y) ) % p == (1 + d*x*x*y*y) % p

def add_points(P: Tuple[int, int], Q: Tuple[int, int]): 
    x1, y1 = P
    x2, y2 = Q
    
    x3 = (((x1*y2 + y1*x2) % p) * pow(1+d*x1*x2*y1*y2, -1, p)) % p
    y3 = (((y1*y2 - a*x1*x2) % p) * pow(1- d*x1*x2*y1*y2, -1, p)) % p
    
    is_on_curve((x3, y3), p)
    
    return x3, y3
    
def apply_double_and_add_method(G: Tuple[int, int], k: int):
    target_point = G
    
    k_binary = bin(k)[2:] #0b1111111001
    
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
        
        # doubling - always
        target_point = add_points(target_point, target_point)
        
        if current_bit == "1":
            target_point = add_points(target_point, G)
    
    is_on_curve(target_point, p)
    
    return target_point

In [3]:
# curve25519
p = pow(2, 255) - 19

# curve arguments for curve25519
a = -1
# d = -121665/121666
d = -121665 * pow(121666, -1, p)

# base point for curve25519
G = (15112221349535400772501151409588531511454012693041857206046113283949847762202
, 46316835694926478169428394003475163141307993866256225615783033603165251855960)

In [4]:
is_on_curve(G, p)

# Alice generates her private and public key pair

In [5]:
# Alice's private key
ka = random.getrandbits(256)

# Alice's public key
Qa = apply_double_and_add_method(G = G, k = ka)

In [6]:
ka

66249982678655119476500490223593026659932278173556590885271747963233235118105

In [7]:
Qa

(1005181306950165921473664928160402336356482915719284029317782011199811421698,
 11155084437648835951367715742496979813363072105257144906550384866492421225154)

# Bob generates a random integer and calculates 2 points

In [8]:
# Bob's random key
rb = random.getrandbits(256)

# send point R to Alice
R = apply_double_and_add_method(G = G, k = rb)

# keep point S secret and use it for symmetric encryption
S = apply_double_and_add_method(G = Qa, k = rb)

In [9]:
rb

104472488918774060848944047765065040208372829090255552015623908500628247701762

In [10]:
R

(7212105680152557366601139353728729848125686331499091131399543418853927346320,
 50172703301935435609180632539621854881248044128731854794076177553673352279286)

In [11]:
S

(5928721286767736012656058470842491468042361782544900814876074735103986730326,
 46643010645291019550172702422507582065206154375688286175779380030276146704100)

# Bob sends point R to Alice

In [12]:
# Alice receives R and finds ka x R
S_prime = apply_double_and_add_method(G = R, k = ka)

In [13]:
S_prime

(5928721286767736012656058470842491468042361782544900814876074735103986730326,
 46643010645291019550172702422507582065206154375688286175779380030276146704100)

In [14]:
assert S == S_prime

# symmetric key encryption (Bob)

In [15]:
msg = "attack tomorrow!"

In [16]:
obj_bob = AES.new(str(S)[0:32])

In [17]:
ciphertext = base64.b64encode(obj_bob.encrypt(msg))

In [18]:
ciphertext

b'Iola5ayKWzcCgx9xG+G/Sw=='

# symmetric key decryption (Alice)

In [19]:
obj_alice = AES.new(str(S)[0:32])

In [20]:
plaintext = obj_alice.decrypt(base64.b64decode(ciphertext))

In [21]:
plaintext

b'attack tomorrow!'